In [79]:
from models import SimilarityPredictorKMeans, SimilarityPredictorBIRCH, ObjectsTextSimilarity, ObjectsSimilarity
import pandas as pd
import json
import pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [80]:
train_data_enc = pd.read_csv("data//train_data_recipes_encoded.csv")
train_data_text = pd.read_csv("data//train_data_text_url.csv")
recipes_url, train_data_text = train_data_text["URL"], train_data_text.drop(columns=["URL"])

train_data_all = train_data_enc.copy()
train_data_all[["Directions", "Ingr"]] = train_data_text

with open("data//recipes_val_indexes.txt", "r") as fp:
    train_val_indx = json.load(fp)

In [81]:
model1 = SimilarityPredictorKMeans()
model1.fit(train_data_enc)
model2 = SimilarityPredictorBIRCH()
model2.fit(train_data_enc)
model3 = ObjectsTextSimilarity()
model3.fit(train_data_text)
model4 = ObjectsSimilarity()
model4.fit(train_data_all, ["Directions", "Ingr"])

C:\Users\dasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [82]:
def validation(models, data):
    val_dir = {}
    for recipe_ind in train_val_indx.keys():
        temp = {}
        for model, dat in zip(models, data):
            if model.__class__.__name__ == "ObjectsTextSimilarity":
                temp[str(model.__class__.__name__)] = list(model.predict(dat.iloc[int(recipe_ind)].values))
            else:
                temp[str(model.__class__.__name__)] = list(model.predict(dat.iloc[int(recipe_ind)]))
        val_dir[recipe_ind] = temp

    return val_dir

results_dir = validation([model1, model2, model3, model4], [train_data_enc, train_data_enc, train_data_text, train_data_all])

In [74]:
print("Recipes for validation: \n\n")
for key in train_val_indx.keys():
    print("Request: ")
    print(recipes_url[int(key)], "\n")
    print(recipes_url.iloc[train_val_indx[key]].values, "\n\n")

Recipes for validation: 


Request: 
https://www.allrecipes.com/recipe/230873/amazing-and-easy-chicken-wings/ 

['https://www.allrecipes.com/recipe/25505/mahogany-chicken-wings/'
 'https://www.allrecipes.com/recipe/213068/grill-master-chicken-wings/'
 'https://www.allrecipes.com/recipe/282258/grilled-sweet-and-sour-chicken-wings/'
 'https://www.allrecipes.com/recipe/218350/simple-marinated-chicken-wings/'
 'https://www.allrecipes.com/recipe/219107/sweet-hot-mustard-chicken-wings/'
 'https://www.allrecipes.com/recipe/267927/sweet-heat-chicken-wings/'
 'https://www.allrecipes.com/recipe/274652/garlic-molasses-chicken-wings/'
 'https://www.allrecipes.com/recipe/214464/chinese-chicken-wings/'
 'https://www.allrecipes.com/recipe/25187/easy-baked-chicken-wings/'
 'https://www.allrecipes.com/recipe/232335/honey-garlic-chicken-wings/'] 


Request: 
https://www.allrecipes.com/recipe/205189/1-pumpkin-spice-cookies/ 

['https://www.allrecipes.com/recipe/24422/chocolate-chip-pumpkin-cookies/'
 'ht

In [75]:
print("Predictions for requests\n\n")

def print_val(dir):
    for recipe_ind in dir.keys():
        print("Request: ")
        print(recipes_url[int(recipe_ind)])
        print()
        for model_key in dir[recipe_ind].keys():
            print(model_key)          
            print(recipes_url.iloc[dir[recipe_ind][model_key]].values)
            print()
        print()

print_val(results_dir)

Predictions for requests


Request: 
https://www.allrecipes.com/recipe/230873/amazing-and-easy-chicken-wings/

SimilarityPredictorKMeans
['https://www.allrecipes.com/recipe/278533/funeral-sandwiches/'
 'https://www.allrecipes.com/recipe/25311/vegetarian-moussaka/'
 'https://www.allrecipes.com/recipe/219880/white-chili-vi/'
 'https://www.allrecipes.com/recipe/217840/papas-rellenas-fried-stuffed-potatoes/'
 'https://www.allrecipes.com/recipe/155328/sweet-pork-slow-cooker-chili/'
 'https://www.allrecipes.com/recipe/262607/chicken-and-lentil-soup-with-homemade-miso/'
 'https://www.allrecipes.com/recipe/279749/shrimp-macaroni-salad/'
 'https://www.allrecipes.com/recipe/73649/chicken-feta-cheese-and-sun-dried-tomato-wraps/'
 'https://www.allrecipes.com/recipe/285933/easy-slow-cooker-tuna-casserole/'
 'https://www.allrecipes.com/recipe/278354/slow-cooker-sweet-potato-and-chorizo-chili/']

SimilarityPredictorBIRCH
['https://www.allrecipes.com/recipe/278533/funeral-sandwiches/'
 'https://www.al

In [76]:
res = [0, 0, 0, 0]
for rec_ind in results_dir.keys():
    for i, model in enumerate(results_dir[rec_ind].keys()):
        res[i] += len(set(train_val_indx[rec_ind]) & set(results_dir[rec_ind][model]))

In [77]:
alg = [
    "Kmeans clustering",
    "BIRCH clustering",
    "ObjectsTextSimilarity (using only Directions and Ingredients features)",
    "ObjectsSimilarity (using all features)"
]

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({"Algorithm": alg, "Accuracy, %": res})
df

,Algorithm,"Accuracy, %"
0,Kmeans clustering,0
1,BIRCH clustering,1
2,ObjectsTextSimilarity (using only Directions and Ingredients features),45
3,ObjectsSimilarity (using all features),0


In [ ]:
file_name = 'ObjectsTextSimilarityModel.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(model3, file)
    print(f'Object successfully saved to "{file_name}"')